In [ ]:
import json 
import time 
import struct 
import sqlalchemy 
import pyodbc 
import notebookutils 
import pandas as pd 
from pyspark.sql import functions as fn 
from datetime import datetime 
import sempy.fabric as fabric 
from sempy.fabric.exceptions import FabricHTTPException, WorkspaceNotFoundException 

def pad_or_truncate_string(input_string, length, pad_char=' '):
    # Truncate if the string is longer than the specified length
    if len(input_string) > length:
        return input_string[:length]
    # Pad if the string is shorter than the specified length
    return input_string.ljust(length, pad_char)

def display_return(data):
    table_details = [
        {
        'tableName': table['tableName'],
        'status':  table['status'],
        'startDateTime':  table['startDateTime'],
        'endDateTime':  table['endDateTime'],
        'lastSuccessfulSyncDateTime':  table['lastSuccessfulSyncDateTime'],
        'error':  table['error']
        }
        for table in data
        ]
    for detail in table_details:
        print(f"Table: {pad_or_truncate_string(detail['tableName'],20)} status: {detail['status']} start: {detail['startDateTime']}  end: {detail['endDateTime']}  Last Update: {detail['lastSuccessfulSyncDateTime']}  error: {detail['error']}  ")
    print('')
    #print(data)
    
    return;

tenant_id=spark.conf.get("trident.tenant.id")
workspace_id=spark.conf.get("trident.workspace.id")
lakehouse_id=spark.conf.get("trident.lakehouse.id")
lakehouse_name=spark.conf.get("trident.lakehouse.name")

#Instantiate the client
client = fabric.FabricRestClient()

# This is the SQL endpoint I want to sync with the lakehouse, this needs to be the GUI
sqlendpoint = fabric.FabricRestClient().get(f"/v1/workspaces/{workspace_id}/lakehouses/{lakehouse_id}").json()['properties']['sqlEndpointProperties']['id']
j_son = fabric.FabricRestClient().get(f"/v1/workspaces/{workspace_id}/lakehouses/{lakehouse_id}").json()
display(sqlendpoint)
 

# URI for the call 
uri = f"v1/workspaces/{workspace_id}/sqlEndpoints/{sqlendpoint}/refreshMetadata" 

# This is the action, we want to take 
payload = {} 
# Payload for a specific table
#payload = { "tableDefinitions": [  {"schema": "dbo", "tableNames": ["demotable"]} ] } 

# Call the REST API 
display(f"uri: {uri}") 

try:
    response = client.post(uri,json= payload) 

    ## You should add some error handling here 

    match response.status_code: 

        case 200: # The request completed in under 30 seconds so it's treated as synchronous. 

            data = json.loads(response.text) 
            display(f"200:") 
            display_return(data)
            display(response.text)

        case 202: # The request took longer than 30 seconds so it's treated as asynchronous. 

            operation_id = response.headers["x-ms-operation-id"] 
            retry_delay = int(response.headers["retry-after"] )
            lro_uri = f"v1/operations/{operation_id}" 
            lro_result_uri = f"{lro_uri}/result" 
        
            display(f"202:") 
                
            #display(f"retry_delay:{retry_delay}")

            while True: 

                lro_response = client.get(lro_uri) 
                lro_data = json.loads(lro_response.text) 

                display(lro_uri)
                #display(lro_response)
                display(lro_response.text)
                #display(lro_data)

                if lro_data["status"] == "Succeeded": 
                    lro_result_response = client.get(lro_result_uri) 
                    lro_result_data = json.loads(lro_result_response.text) 
                    print(f"202:") 
                    display(lro_result_data)
                    display(lro_result_response.text)
                    break 
                if lro_data["status"]  == "Failed": 
                    print(f"Failed 202: {lro_data}") 
                    break 

                
                print(f"waiting...") 
                time.sleep(retry_delay) 
        case 500:
                print(f"The sync is already running...") 
                display(response.text)
        case _:
            data = json.loads(response.text) 
            print(f"case else:") 
            display_return(data)
            display(response.text)

except Exception as e: print(e)